In [ ]:
import tkinter as tk
import numpy as np
import random
import time
from queue import Queue, PriorityQueue
from collections import deque

# Initialize global variables
GRID_SIZE = 20
CANVAS_SIZE = 600
BUTTON_HEIGHT = 50

# Maze sizes for small, medium, and large
SMALL_MAZE = 10
MEDIUM_MAZE = 20
LARGE_MAZE = 30

# 8-Puzzle Goal State
PUZZLE_GOAL_STATE = [1, 2, 3, 4, 5, 6, 7, 8, 0]  # 0 represents the empty tile

class MazeSolverApp:
    def __init__(self, root):
        self.root = root
        self.root.title("AI Search Problem Solver")

        # Create canvas for problems
        self.canvas = tk.Canvas(self.root, width=CANVAS_SIZE, height=CANVAS_SIZE, bg='white')
        self.canvas.grid(row=0, column=0, columnspan=6)

        # Control buttons
        self.random_button = tk.Button(self.root, text="Random Maze", command=self.generate_random_maze)
        self.random_button.grid(row=1, column=0)

        self.dfs_button = tk.Button(self.root, text="Solve with DFS", command=self.solve_with_dfs)
        self.dfs_button.grid(row=1, column=1)

        self.bfs_button = tk.Button(self.root, text="Solve with BFS", command=self.solve_with_bfs)
        self.bfs_button.grid(row=1, column=2)

        self.astar_button = tk.Button(self.root, text="Solve with A*", command=self.solve_with_astar)
        self.astar_button.grid(row=1, column=3)

        self.bidirectional_button = tk.Button(self.root, text="Solve with Bidirectional", command=self.solve_with_bidirectional)
        self.bidirectional_button.grid(row=1, column=4)

        self.problem_var = tk.StringVar(value="Maze")
        self.problem_menu = tk.OptionMenu(self.root, self.problem_var, "Maze", "String Transformation", "8-Puzzle", command=self.change_problem)
        self.problem_menu.grid(row=1, column=5)

        # Set the default problem type and initialize
        self.problem_type = "Maze"
        self.solution_path = None
        self.explored_path = []

        self.generate_random_maze()

    def change_problem(self, problem):
        self.problem_type = problem
        if problem == "Maze":
            self.generate_random_maze()
        elif problem == "String Transformation":
            self.generate_string_problem()
        elif problem == "8-Puzzle":
            self.generate_puzzle_problem()

    # Maze Problem
    def generate_random_maze(self):
        self.maze_size = SMALL_MAZE
        self.maze = self.generate_maze(self.maze_size)
        self.draw_maze()

    def generate_maze(self, size):
        maze = np.ones((size, size), dtype=int)
        maze[0][0] = 0
        maze[size - 1][size - 1] = 0

        def dfs_generate(x, y):
            directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
            random.shuffle(directions)

            for dx, dy in directions:
                nx, ny = x + dx, y + dy
                if 0 <= nx < size and 0 <= ny < size and maze[nx][ny] == 1:
                    maze[nx][ny] = 0
                    dfs_generate(nx, ny)

        dfs_generate(0, 0)
        return maze

    def draw_maze(self):
        self.canvas.delete("all")
        size = len(self.maze)
        cell_size = CANVAS_SIZE // size
        for i in range(size):
            for j in range(size):
                color = 'white' if self.maze[i][j] == 0 else 'black'
                if (i, j) == (0, 0):
                    color = 'green'
                elif (i, j) == (size - 1, size - 1):
                    color = 'red'
                self.canvas.create_rectangle(j * cell_size, i * cell_size,
                                             (j + 1) * cell_size, (i + 1) * cell_size, fill=color)

    # String Transformation Problem
    def generate_string_problem(self):
        self.start_word = "cat"
        self.end_word = "dog"
        self.valid_words = {"cat", "bat", "bot", "bog", "dog", "cog", "cot", "dot"}
        self.solution_path = []
        self.display_string_problem()

    def display_string_problem(self):
        self.canvas.delete("all")
        self.canvas.create_text(CANVAS_SIZE / 2, CANVAS_SIZE / 3, text=f"Transform '{self.start_word}' to '{self.end_word}'", font=("Arial", 20))

    def solve_string_problem(self):
        self.solution_path = bfs_string(self.start_word, self.end_word, self.valid_words)
        print(f"Solution: {self.solution_path}")

    # 8-Puzzle Problem
    def generate_puzzle_problem(self):
        self.puzzle_start = [1, 2, 3, 4, 5, 6, 0, 7, 8]
        self.puzzle_goal = PUZZLE_GOAL_STATE
        self.solution_path = []
        self.display_puzzle()

    def display_puzzle(self):
        self.canvas.delete("all")
        size = int(len(self.puzzle_start) ** 0.5)
        cell_size = CANVAS_SIZE // size
        for i in range(size):
            for j in range(size):
                value = self.puzzle_start[i * size + j]
                color = 'white' if value != 0 else 'black'
                self.canvas.create_rectangle(j * cell_size, i * cell_size,
                                             (j + 1) * cell_size, (i + 1) * cell_size, fill=color)
                if value != 0:
                    self.canvas.create_text(j * cell_size + cell_size / 2, i * cell_size + cell_size / 2,
                                            text=str(value), font=("Arial", 20))

    def solve_with_dfs(self):
        self.solve_problem(dfs, "DFS")

    def solve_with_bfs(self):
        self.solve_problem(bfs, "BFS")

    def solve_with_astar(self):
        self.solve_problem(a_star, "A*")

    def solve_with_bidirectional(self):
        self.solve_problem(bidirectional_search, "Bidirectional Search")

    def solve_problem(self, algorithm, name):
        start_time = time.time()
        if self.problem_type == "Maze":
            self.explored_path, self.solution_path = algorithm(self.maze)
        elif self.problem_type == "String Transformation":
            self.solve_string_problem()
        elif self.problem_type == "8-Puzzle":
            self.solution_path = bfs_puzzle(self.puzzle_start, self.puzzle_goal)
        elapsed_time = time.time() - start_time
        print(f"{name} solved the problem in {elapsed_time:.4f} seconds.")

# BFS for String Transformation
def bfs_string(start, end, valid_words):
    queue = Queue()
    queue.put([start])
    visited = set([start])

    while not queue.empty():
        path = queue.get()
        word = path[-1]

        if word == end:
            return path

        for next_word in valid_words:
            if next_word not in visited and sum(a != b for a, b in zip(word, next_word)) == 1:
                visited.add(next_word)
                queue.put(path + [next_word])

    return None

# BFS for 8-Puzzle
def bfs_puzzle(start, goal):
    # Implementation of BFS for the 8-puzzle problem
    pass  # Complete as needed

# Main execution
if __name__ == "__main__":
    root = tk.Tk()
    app = MazeSolverApp(root)
    root.mainloop()
